#### Loading All the Necessary Libararies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow import keras
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras import layers
import string

[nltk_data] Downloading package stopwords to /home/kvp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


#### Reading the training data

In [2]:
import os,glob
folder_path ='MT'
i=0
MT_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        MT_sents.extend(f.read().split("\n"))
MT_sents=list(set(MT_sents))

folder_path ='OC'
i=0
OC_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        OC_sents.extend(f.read().split("\n"))
OC_sents=list(set(OC_sents))

folder_path ='PE'
i=0
PE_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        PE_sents.extend(f.read().split("\n"))
PE_sents=list(set(PE_sents))
        
folder_path ='VG'
i=0
VG_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        VG_sents.extend(f.read().split("\n"))
VG_sents=list(set(VG_sents))
        
folder_path ='WD'
i=0
WD_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WD_sents.extend(f.read().split("\n"))
WD_sents=list(set(WD_sents))

folder_path ='WTP'
i=0
WTP_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WTP_sents.extend(f.read().split("\n"))
WTP_sents=list(set(WTP_sents))

In [3]:
print(len(MT_sents))
print(len(OC_sents))
print(len(PE_sents))
print(len(VG_sents))
print(len(WD_sents))
print(len(WTP_sents))

450
8529
7052
2577
3873
8411


#### Preprocessing the data

In [4]:
MT_s=[]
MT_l=[]
for i in MT_sents:
    if len(i)>1:
        MT_s.append(i.split("\t")[0])
        MT_l.append(int(i.split("\t")[1]))

        
OC_s=[]
OC_l=[]
for i in OC_sents:
    if len(i)>1:
        OC_s.append(i.split("\t")[0])
        OC_l.append(int(i.split("\t")[1]))

        
PE_s=[]
PE_l=[]
for i in PE_sents:
    if len(i)>1:
        PE_s.append(i.split("\t")[0])
        PE_l.append(int(i.split("\t")[1]))
        

VG_s=[]
VG_l=[]
for i in VG_sents:
    if len(i)>1:
        VG_s.append(i.split("\t")[0])
        VG_l.append(int(i.split("\t")[1]))
        
        
WD_s=[]
WD_l=[]
for i in WD_sents:
    if len(i)>1:
        WD_s.append(i.split("\t")[0])
        WD_l.append(int(i.split("\t")[1]))
        
        
WTP_s=[]
WTP_l=[]
for i in WTP_sents:
    if len(i)>1:
        WTP_s.append(i.split("\t")[0])
        WTP_l.append(int(i.split("\t")[1]))

#### Removing the punctuation and cleaning the data

In [5]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text


##### Converting the dataset into a Pandas dataframe

In [6]:
df_MT=pd.DataFrame({'sentence':MT_s,'label':MT_l})
df_OC=pd.DataFrame({'sentence':OC_s,'label':OC_l})
df_PE=pd.DataFrame({'sentence':PE_s,'label':PE_l})
df_VG=pd.DataFrame({'sentence':VG_s,'label':VG_l})
df_WD=pd.DataFrame({'sentence':WD_s,'label':WD_l})
df_WTP=pd.DataFrame({'sentence':WTP_s,'label':WTP_l})

In [16]:
df_MT['sentence'] = df_MT['sentence'].map(lambda x: clean_text(x))
df_OC['sentence'] = df_OC['sentence'].map(lambda x: clean_text(x))
df_PE['sentence'] = df_PE['sentence'].map(lambda x: clean_text(x))
df_VG['sentence'] = df_VG['sentence'].map(lambda x: clean_text(x))
df_WD['sentence'] = df_WD['sentence'].map(lambda x: clean_text(x))
df_WTP['sentence'] = df_WTP['sentence'].map(lambda x: clean_text(x))

#### Showing the data

In [8]:
df_MT.head()

,sentence,label
0,never longer queue advanc ticket offic japan o...,0
1,patient could learn awar bodi treat healthili,0
2,everi day passeng harass hooligan drunk underg...,0
3,everyon must given chanc hone conscienc possib...,0
4,good overal grade open mani door top prioriti ...,0


In [10]:
df_OC.head()

,sentence,label
0,matter goal say odd make much ground toward goal,0
1,friend,0
2,liz sorri,0
3,anyon els invit gollancz quiz night tonight,0
4,hahah,0


In [12]:
df_VG.head()

,sentence,label
0,howard north asian tour one oversea visit take...,1
1,second britain militari stand america current ...,0
2,therefor cambridg astronom fred hoyl point big...,1
3,deliv public privat messag urg dictat leav ira...,0
4,cours also possibl christian believ afterlif g...,0


In [13]:
df_PE.head()

,sentence,label
0,point view discuss order,0
1,student also learn sourc experi librari peopl,0
2,amount control media inform,0
3,learn econom everi thing want,0
4,furthermor femal student highli interest learn...,1


In [14]:
df_WD.head()

,sentence,label
0,interest cover subject learn much quicker inte...,0
1,everyon know went public school colleg,0
2,parent lead exampl,0
3,know sound wrong way,0
4,,0


In [15]:
df_WTP.head()

,sentence,label
0,sentenc place make sens,1
1,current await arriv annuario pontificio see co...,0
2,could calm person commentari,0
3,interest threat previous point gap employe ent...,1
4,previous display imag vanita philipp champaign...,0


#### In-Domain Experiments for CNN model

In [62]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#### Since the dataset is not well balanced. We train 20 models where in each model we discard certain samples of label 0 to make the dataset of equal size. We repeat this process for subsequent datasets

#### 1. On MT dataset

In [112]:
df_positive=df_MT[df_MT["label"]==1]
df_negative=df_MT[df_MT["label"]==0]
print(len(df_negative))
print(len(df_positive))

337
112


In [113]:
f1_MT=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_MT[df_MT["label"]==1]
    df_negative=df_MT[df_MT["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_MT.append(report["macro avg"]["f1-score"])

Model 1
Epoch 1/15
201/201 [==============================] - 2s 12ms/step - loss: 0.6971 - accuracy: 0.4726
Epoch 2/15
201/201 [==============================] - 0s 2ms/step - loss: 0.6405 - accuracy: 0.8756
Epoch 3/15
201/201 [==============================] - 0s 2ms/step - loss: 0.5536 - accuracy: 0.9502
Epoch 4/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3937 - accuracy: 0.9801
Epoch 5/15
201/201 [==============================] - 0s 2ms/step - loss: 0.1887 - accuracy: 0.9801
Epoch 6/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0629 - accuracy: 0.9950
Epoch 7/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0240 - accuracy: 0.9950
Epoch 8/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 1.0000
Epoch 9/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 10/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0067 - accurac

Epoch 7/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3297 - accuracy: 0.9950
Epoch 8/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3205 - accuracy: 0.9950
Epoch 9/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3134 - accuracy: 0.9950
Epoch 10/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3091 - accuracy: 1.0000
Epoch 11/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3028 - accuracy: 0.9950
Epoch 12/15
201/201 [==============================] - 0s 2ms/step - loss: 0.2966 - accuracy: 1.0000
Epoch 13/15
201/201 [==============================] - 0s 2ms/step - loss: 0.2917 - accuracy: 1.0000
Epoch 14/15
201/201 [==============================] - 0s 2ms/step - loss: 0.2869 - accuracy: 1.0000
Epoch 15/15
23/23 [==============================] - 8s 343ms/step
              precision    recall  f1-score   support

           0       1.00      0.07      0.13        14
           1       

Epoch 13/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 14/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 15/15
23/23 [==============================] - 8s 350ms/step
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        14
           1       0.56      1.00      0.72         9

    accuracy                           0.70        23
   macro avg       0.78      0.75      0.69        23
weighted avg       0.83      0.70      0.69        23

Model 10
Epoch 1/15
201/201 [==============================] - 3s 15ms/step - loss: 0.6937 - accuracy: 0.4776
Epoch 2/15
201/201 [==============================] - 0s 2ms/step - loss: 0.6505 - accuracy: 0.7861
Epoch 3/15
201/201 [==============================] - 0s 2ms/step - loss: 0.5739 - accuracy: 0.8259
Epoch 4/15
201/201 [==============================] - 0s 2ms/step - loss: 0.4211 - accuracy: 0

Epoch 1/15
201/201 [==============================] - 3s 15ms/step - loss: 0.6939 - accuracy: 0.5025
Epoch 2/15
201/201 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 0.9104
Epoch 3/15
201/201 [==============================] - 0s 2ms/step - loss: 0.5695 - accuracy: 0.9303
Epoch 4/15
201/201 [==============================] - 0s 2ms/step - loss: 0.3920 - accuracy: 0.9851
Epoch 5/15
201/201 [==============================] - 0s 2ms/step - loss: 0.1698 - accuracy: 0.9950
Epoch 6/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0470 - accuracy: 1.0000
Epoch 7/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 8/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 9/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 10/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.000

Epoch 7/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 8/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 9/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 10/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 11/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 12/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 13/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 14/15
201/201 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 15/15
23/23 [==============================] - 9s 372ms/step
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        14
           1       

In [114]:
print(f"Macro F1 score of the trained model is {sum(f1_MT)/len(f1_MT)}")  

Macro F1 score of the trained model is 0.6706580001283624


#### 2. On OC dataset

In [66]:
df_positive=df_OC[df_OC["label"]==1]
df_negative=df_OC[df_OC["label"]==0]
print(len(df_negative))
print(len(df_positive))

7826
702


In [68]:
f1_OC=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_OC[df_OC["label"]==1]
    df_negative=df_OC[df_OC["label"]==0]
    df_negative=df_negative.sample(n=702)
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_OC.append(report["macro avg"]["f1-score"])

Model 1
Epoch 1/15
1263/1263 [==============================] - 2s 2ms/step - loss: 0.6870 - accuracy: 0.5550
Epoch 2/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.6045 - accuracy: 0.6968
Epoch 3/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.3408 - accuracy: 0.8527
Epoch 4/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.1380 - accuracy: 0.9477
Epoch 5/15
1263/1263 [==============================] - 2s 1ms/step - loss: 0.0756 - accuracy: 0.9770
Epoch 6/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0518 - accuracy: 0.9810
Epoch 7/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0437 - accuracy: 0.9802
Epoch 8/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0389 - accuracy: 0.9842
Epoch 9/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0355 - accuracy: 0.9834
Epoch 10/15
1263/1263 [==============================] - 1s 1ms/step - los

1263/1263 [==============================] - 1s 1ms/step - loss: 0.0263 - accuracy: 0.9889
Epoch 10/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0214 - accuracy: 0.9873
Epoch 11/15
1263/1263 [==============================] - 2s 1ms/step - loss: 0.0196 - accuracy: 0.9897
Epoch 12/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0176 - accuracy: 0.9913
Epoch 13/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0182 - accuracy: 0.9913
Epoch 14/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0186 - accuracy: 0.9921
Epoch 15/15
141/141 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

           0       0.49      0.49      0.49        75
           1       0.42      0.42      0.42        66

    accuracy                           0.46       141
   macro avg       0.46      0.46      0.46       141
weighted avg       0.46      0.46      0.46       141


Epoch 1/15
1263/1263 [==============================] - 2s 2ms/step - loss: 0.6780 - accuracy: 0.5859
Epoch 2/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.5647 - accuracy: 0.7308
Epoch 3/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.2945 - accuracy: 0.8907
Epoch 4/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.1335 - accuracy: 0.9477
Epoch 5/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0742 - accuracy: 0.9747
Epoch 6/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0476 - accuracy: 0.9834
Epoch 7/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0383 - accuracy: 0.9857
Epoch 8/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0346 - accuracy: 0.9865
Epoch 9/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.0326 - accuracy: 0.9865
Epoch 10/15
1263/1263 [==============================] - 1s 1ms/step - loss: 0.027

In [69]:
print(f"Macro F1 score of the trained model is {sum(f1_OC)/len(f1_OC)}")  

Macro F1 score of the trained model is 0.512757318649584


#### 3. On the VG dataset

In [70]:
df_positive=df_VG[df_VG["label"]==1]
df_negative=df_VG[df_VG["label"]==0]
print(len(df_negative))
print(len(df_positive))

2029
547


In [71]:
f1_VG=[]
for i in range(0,20):
    print(f"Model {i}")
    df_positive=df_VG[df_VG["label"]==1]
    df_negative=df_VG[df_VG["label"]==0]
    df_negative=df_negative.sample(n=547)
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=30

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_VG.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/15
984/984 [==============================] - 2s 2ms/step - loss: 0.6891 - accuracy: 0.5508
Epoch 2/15
984/984 [==============================] - 1s 1ms/step - loss: 0.5717 - accuracy: 0.7510
Epoch 3/15
984/984 [==============================] - 1s 1ms/step - loss: 0.2640 - accuracy: 0.9157
Epoch 4/15
984/984 [==============================] - 1s 974us/step - loss: 0.0899 - accuracy: 0.9726
Epoch 5/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0515 - accuracy: 0.9837
Epoch 6/15
984/984 [==============================] - 1s 996us/step - loss: 0.0330 - accuracy: 0.9888
Epoch 7/15
984/984 [==============================] - 1s 984us/step - loss: 0.0308 - accuracy: 0.9909
Epoch 8/15
984/984 [==============================] - 1s 999us/step - loss: 0.0304 - accuracy: 0.9909
Epoch 9/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0234 - accuracy: 0.9929
Epoch 10/15
984/984 [==============================] - 1s 992us/step - loss: 0.0226 

984/984 [==============================] - 1s 833us/step - loss: 0.0331 - accuracy: 0.9929
Epoch 7/15
984/984 [==============================] - 1s 822us/step - loss: 0.0280 - accuracy: 0.9919
Epoch 8/15
984/984 [==============================] - 1s 825us/step - loss: 0.0264 - accuracy: 0.9939
Epoch 9/15
984/984 [==============================] - 1s 826us/step - loss: 0.0194 - accuracy: 0.9949
Epoch 10/15
984/984 [==============================] - 1s 825us/step - loss: 0.0169 - accuracy: 0.9959
Epoch 11/15
984/984 [==============================] - 1s 827us/step - loss: 0.0198 - accuracy: 0.9949
Epoch 12/15
984/984 [==============================] - 1s 831us/step - loss: 0.0206 - accuracy: 0.9939
Epoch 13/15
984/984 [==============================] - 1s 818us/step - loss: 0.0159 - accuracy: 0.9939
Epoch 14/15
984/984 [==============================] - 1s 818us/step - loss: 0.0177 - accuracy: 0.9949
Epoch 15/15
110/110 [==============================] - 2s 17ms/step
              precis

Epoch 1/15
984/984 [==============================] - 2s 2ms/step - loss: 0.6930 - accuracy: 0.5020
Epoch 2/15
984/984 [==============================] - 1s 1ms/step - loss: 0.6489 - accuracy: 0.7144
Epoch 3/15
984/984 [==============================] - 1s 1ms/step - loss: 0.3279 - accuracy: 0.8913
Epoch 4/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0901 - accuracy: 0.9726
Epoch 5/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0470 - accuracy: 0.9868
Epoch 6/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0402 - accuracy: 0.9919
Epoch 7/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0285 - accuracy: 0.9939
Epoch 8/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0286 - accuracy: 0.9929
Epoch 9/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0285 - accuracy: 0.9919
Epoch 10/15
984/984 [==============================] - 1s 1ms/step - loss: 0.0244 - accuracy: 0.9939

In [72]:
print(f"Macro F1 score of the trained model is {sum(f1_VG)/len(f1_VG)}")  

Macro F1 score of the trained model is 0.5455825444241291


#### On the PE dataset

In [90]:
df_positive=df_PE[df_PE["label"]==1]
df_negative=df_PE[df_PE["label"]==0]
print(len(df_negative))
print(len(df_positive))

4958
2093


In [107]:
f1_PE=[]
for i in range(0,20):
    print(f"Model {i}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=2093)
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_PE.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/15
3767/3767 [==============================] - 11s 3ms/step - loss: 0.6307 - accuracy: 0.6419
Epoch 2/15
3767/3767 [==============================] - 8s 2ms/step - loss: 0.4330 - accuracy: 0.8102
Epoch 3/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.2157 - accuracy: 0.9246
Epoch 4/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.0908 - accuracy: 0.9729
Epoch 5/15
3767/3767 [==============================] - 9s 3ms/step - loss: 0.0379 - accuracy: 0.9904
Epoch 6/15
3767/3767 [==============================] - 8s 2ms/step - loss: 0.0254 - accuracy: 0.9944
Epoch 7/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.0162 - accuracy: 0.9971
Epoch 8/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0117 - accuracy: 0.9976
Epoch 9/15
3767/3767 [==============================] - 10s 3ms/step - loss: 0.0095 - accuracy: 0.9976
Epoch 10/15
3767/3767 [==============================] - 7s 2ms/step - l

3767/3767 [==============================] - 7s 2ms/step - loss: 0.0055 - accuracy: 0.9989
Epoch 10/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.0069 - accuracy: 0.9995
Epoch 11/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.0058 - accuracy: 0.9995
Epoch 12/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0039 - accuracy: 0.9997
Epoch 13/15
3767/3767 [==============================] - 7s 2ms/step - loss: 0.0052 - accuracy: 0.9995
Epoch 14/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0029 - accuracy: 0.9992
Epoch 15/15
419/419 [==============================] - 9s 22ms/step
              precision    recall  f1-score   support

           0       0.60      0.71      0.65       192
           1       0.71      0.60      0.65       227

    accuracy                           0.65       419
   macro avg       0.66      0.66      0.65       419
weighted avg       0.66      0.65      0.65       419


Epoch 1/15
3767/3767 [==============================] - 8s 2ms/step - loss: 0.6504 - accuracy: 0.5968
Epoch 2/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.4754 - accuracy: 0.7805
Epoch 3/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.2581 - accuracy: 0.9031
Epoch 4/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.1107 - accuracy: 0.9658
Epoch 5/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0459 - accuracy: 0.9867
Epoch 6/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0217 - accuracy: 0.9971
Epoch 7/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0103 - accuracy: 0.9987
Epoch 8/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0060 - accuracy: 0.9992
Epoch 9/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.0044 - accuracy: 0.9997
Epoch 10/15
3767/3767 [==============================] - 6s 2ms/step - loss: 0.004

In [108]:
print(f"Macro F1 score of the trained model is {sum(f1_PE)/len(f1_PE)}")  

Macro F1 score of the trained model is 0.6708510740116682


#### On the WD dataset

In [75]:
df_positive=df_WD[df_WD["label"]==1]
df_negative=df_WD[df_WD["label"]==0]
print(len(df_negative))
print(len(df_positive))

3661
211


In [76]:
f1_WD=[]
for i in range(0,20):
    print(f"Model {i}")
    df_positive=df_WD[df_WD["label"]==1]
    df_negative=df_WD[df_WD["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_WD.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/15
379/379 [==============================] - 1s 3ms/step - loss: 0.6810 - accuracy: 0.5620
Epoch 2/15
379/379 [==============================] - 0s 1ms/step - loss: 0.5613 - accuracy: 0.7863
Epoch 3/15
379/379 [==============================] - 0s 1ms/step - loss: 0.3583 - accuracy: 0.8734
Epoch 4/15
379/379 [==============================] - 0s 1ms/step - loss: 0.1664 - accuracy: 0.9683
Epoch 5/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0692 - accuracy: 0.9894
Epoch 6/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0337 - accuracy: 0.9947
Epoch 7/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0192 - accuracy: 0.9974
Epoch 8/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0100 - accuracy: 1.0000
Epoch 9/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 10/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy

Epoch 7/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 8/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 9/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 10/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 11/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 12/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 13/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 14/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 15/15
43/43 [==============================] - 3s 59ms/step
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        21
           1       0

379/379 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 14/15
379/379 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 15/15
43/43 [==============================] - 3s 60ms/step
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        21
           1       0.68      0.68      0.68        22

    accuracy                           0.67        43
   macro avg       0.67      0.67      0.67        43
weighted avg       0.67      0.67      0.67        43

Model 9
Epoch 1/15
379/379 [==============================] - 1s 4ms/step - loss: 0.6867 - accuracy: 0.5409
Epoch 2/15
379/379 [==============================] - 0s 1ms/step - loss: 0.5862 - accuracy: 0.8206
Epoch 3/15
379/379 [==============================] - 0s 1ms/step - loss: 0.4016 - accuracy: 0.9103
Epoch 4/15
379/379 [==============================] - 0s 1ms/step - loss: 0.1748 - accuracy: 0.9736
Epoch 5/1

Epoch 1/15
379/379 [==============================] - 1s 3ms/step - loss: 0.6888 - accuracy: 0.5752
Epoch 2/15
379/379 [==============================] - 0s 883us/step - loss: 0.6303 - accuracy: 0.7203
Epoch 3/15
379/379 [==============================] - 0s 926us/step - loss: 0.4573 - accuracy: 0.8707
Epoch 4/15
379/379 [==============================] - 0s 885us/step - loss: 0.2236 - accuracy: 0.9499
Epoch 5/15
379/379 [==============================] - 0s 880us/step - loss: 0.0858 - accuracy: 0.9842
Epoch 6/15
379/379 [==============================] - 0s 895us/step - loss: 0.0417 - accuracy: 0.9921
Epoch 7/15
379/379 [==============================] - 0s 899us/step - loss: 0.0209 - accuracy: 0.9947
Epoch 8/15
379/379 [==============================] - 0s 879us/step - loss: 0.0115 - accuracy: 1.0000
Epoch 9/15
379/379 [==============================] - 0s 895us/step - loss: 0.0072 - accuracy: 1.0000
Epoch 10/15
379/379 [==============================] - 0s 873us/step - loss: 0.0048 

In [77]:
print(f"Macro F1 score of the trained model is {sum(f1_WD)/len(f1_WD)}")  

Macro F1 score of the trained model is 0.6476553773136778


#### On the WTP dataset

In [78]:
df_positive=df_WTP[df_WTP["label"]==1]
df_negative=df_WTP[df_WTP["label"]==0]
print(len(df_negative))
print(len(df_positive))

7275
1135


In [79]:
f1_WTP=[]
for i in range(0,20):
    print(f"Model {i}")
    df_positive=df_WTP[df_WTP["label"]==1]
    df_negative=df_WTP[df_WTP["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y=df["label"].values
    
    sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.1,random_state=1000)

    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_WTP.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/15
2043/2043 [==============================] - 4s 2ms/step - loss: 0.6932 - accuracy: 0.5017
Epoch 2/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.6622 - accuracy: 0.6040
Epoch 3/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.4102 - accuracy: 0.8292
Epoch 4/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.1537 - accuracy: 0.9481
Epoch 5/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0593 - accuracy: 0.9785
Epoch 6/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0282 - accuracy: 0.9892
Epoch 7/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0201 - accuracy: 0.9941
Epoch 8/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0148 - accuracy: 0.9936
Epoch 9/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0153 - accuracy: 0.9946
Epoch 10/15
2043/2043 [==============================] - 3s 1ms/step - los

2043/2043 [==============================] - 3s 1ms/step - loss: 0.0175 - accuracy: 0.9927
Epoch 10/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0135 - accuracy: 0.9931
Epoch 11/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0134 - accuracy: 0.9931
Epoch 12/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0131 - accuracy: 0.9922
Epoch 13/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0114 - accuracy: 0.9946
Epoch 14/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0132 - accuracy: 0.9931
Epoch 15/15
227/227 [==============================] - 3s 14ms/step
              precision    recall  f1-score   support

           0       0.51      0.56      0.53       113
           1       0.51      0.46      0.49       114

    accuracy                           0.51       227
   macro avg       0.51      0.51      0.51       227
weighted avg       0.51      0.51      0.51       227


Epoch 1/15
2043/2043 [==============================] - 4s 2ms/step - loss: 0.6926 - accuracy: 0.5066
Epoch 2/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.6274 - accuracy: 0.6823
Epoch 3/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.3271 - accuracy: 0.8786
Epoch 4/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.1088 - accuracy: 0.9652
Epoch 5/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0467 - accuracy: 0.9829
Epoch 6/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0282 - accuracy: 0.9897
Epoch 7/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0222 - accuracy: 0.9912
Epoch 8/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0198 - accuracy: 0.9922
Epoch 9/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.0232 - accuracy: 0.9902
Epoch 10/15
2043/2043 [==============================] - 3s 1ms/step - loss: 0.021

In [80]:
print(f"Macro F1 score of the trained model is {sum(f1_WTP)/len(f1_WTP)}")  

Macro F1 score of the trained model is 0.5372314333934511


#### Cross Domain Experiments

In [81]:
frames_MT=[df_OC,df_PE,df_VG,df_WD,df_WTP]
frames_OC=[df_MT,df_PE,df_VG,df_WD,df_WTP]
frames_PE=[df_OC,df_MT,df_VG,df_WD,df_WTP]
frames_VG=[df_OC,df_PE,df_MT,df_WD,df_WTP]
frames_WD=[df_OC,df_PE,df_VG,df_MT,df_WTP]
frames_WTP=[df_OC,df_PE,df_VG,df_WD,df_MT]

In [82]:
res_MT=pd.concat(frames_MT)
res_OC=pd.concat(frames_OC)
res_PE=pd.concat(frames_PE)
res_VG=pd.concat(frames_VG)
res_WD=pd.concat(frames_WD)
res_WTP=pd.concat(frames_WTP)

In [83]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

#### Leaving the MT dataset and testing on it

In [84]:
df_positive=res_MT[res_MT["label"]==1]
df_negative=res_MT[res_MT["label"]==0]
print(len(df_negative))
print(len(df_positive))

25749
4688


In [85]:
f1_withoutMT=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_MT[res_MT["label"]==1]
    df_negative=res_MT[res_MT["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_MT['sentence'].values
    y_test=df_MT["label"].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutMT.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
9376/9376 [==============================] - 20s 2ms/step - loss: 0.6395 - accuracy: 0.6221
Epoch 2/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.5008 - accuracy: 0.7660
Epoch 3/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.2652 - accuracy: 0.8971
Epoch 4/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.1093 - accuracy: 0.9609
Epoch 5/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.0537 - accuracy: 0.9824
Epoch 6/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.0383 - accuracy: 0.9877
Epoch 7/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.0354 - accuracy: 0.9894
Epoch 8/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.0309 - accuracy: 0.9902
Epoch 9/10
9376/9376 [==============================] - 19s 2ms/step - loss: 0.0303 - accuracy: 0.9901
Epoch 10/10
449/449 [==============================] - 4s 8ms/ste

In [86]:
print(f"Macro F1 score of the trained model is {sum(f1_withoutMT)/len(f1_withoutMT)}")  

Macro F1 score of the trained model is 0.49936555907628327


#### Leaving the OC dataset and testing on it

In [87]:
df_positive=res_OC[res_OC["label"]==1]
df_negative=res_OC[res_OC["label"]==0]
print(len(df_negative))
print(len(df_positive))

18260
4098


In [88]:
f1_withoutOC=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_OC[res_OC["label"]==1]
    df_negative=res_OC[res_OC["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_OC['sentence'].values
    y_test=df_OC["label"].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutOC.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
8196/8196 [==============================] - 17s 2ms/step - loss: 0.6409 - accuracy: 0.6229
Epoch 2/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.4855 - accuracy: 0.7717
Epoch 3/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.2197 - accuracy: 0.9151
Epoch 4/10
8196/8196 [==============================] - 16s 2ms/step - loss: 0.0765 - accuracy: 0.9768
Epoch 5/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.0340 - accuracy: 0.9891
Epoch 6/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.0200 - accuracy: 0.9939
Epoch 7/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.0196 - accuracy: 0.9935
Epoch 8/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.0180 - accuracy: 0.9941
Epoch 9/10
8196/8196 [==============================] - 15s 2ms/step - loss: 0.0173 - accuracy: 0.9933
Epoch 10/10
8528/8528 [==============================] - 5s 586us

In [89]:
print(f"Macro F1 score of the trained model is {sum(f1_withoutOC)/len(f1_withoutOC)}")  

Macro F1 score of the trained model is 0.5048700080596917


#### Leaving the PE dataset and testing on it

In [90]:
df_positive=res_PE[res_PE["label"]==1]
df_negative=res_PE[res_PE["label"]==0]
print(len(df_negative))
print(len(df_positive))

21128
2707


In [91]:
f1_withoutPE=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_PE[res_PE["label"]==1]
    df_negative=res_PE[res_PE["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_PE['sentence'].values
    y_test=df_PE['label'].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutPE.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
5414/5414 [==============================] - 12s 2ms/step - loss: 0.6731 - accuracy: 0.5824
Epoch 2/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.5427 - accuracy: 0.7322
Epoch 3/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.2633 - accuracy: 0.8931
Epoch 4/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.1032 - accuracy: 0.9647
Epoch 5/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.0585 - accuracy: 0.9793
Epoch 6/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.0435 - accuracy: 0.9839
Epoch 7/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.0388 - accuracy: 0.9854
Epoch 8/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.0362 - accuracy: 0.9867
Epoch 9/10
5414/5414 [==============================] - 10s 2ms/step - loss: 0.0333 - accuracy: 0.9876
Epoch 10/10
7051/7051 [==============================] - 5s 723us

In [92]:
print(f"Macro F1 score of the trained model is {sum(f1_withoutPE)/len(f1_withoutPE)}")  

Macro F1 score of the trained model is 0.516564585934417


#### Leaving the VG dataset and testing pn it

In [93]:
df_positive=res_VG[res_VG["label"]==1]
df_negative=res_VG[res_VG["label"]==0]
print(len(df_negative))
print(len(df_positive))

24057
4253


In [94]:
f1_withoutVG=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_VG[res_VG["label"]==1]
    df_negative=res_VG[res_VG["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_VG['sentence'].values
    y_test=df_VG['label'].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutVG.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
8506/8506 [==============================] - 21s 2ms/step - loss: 0.6589 - accuracy: 0.6226
Epoch 2/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.5380 - accuracy: 0.7371
Epoch 3/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.3412 - accuracy: 0.8597
Epoch 4/10
8506/8506 [==============================] - 20s 2ms/step - loss: 0.1583 - accuracy: 0.9442
Epoch 5/10
8506/8506 [==============================] - 20s 2ms/step - loss: 0.0735 - accuracy: 0.9768
Epoch 6/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.0452 - accuracy: 0.9858
Epoch 7/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.0342 - accuracy: 0.9891
Epoch 8/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.0291 - accuracy: 0.9897
Epoch 9/10
8506/8506 [==============================] - 19s 2ms/step - loss: 0.0272 - accuracy: 0.9904
Epoch 10/10
2576/2576 [==============================] - 5s 2ms/s

In [95]:
print(f"Macro F1 score of the trained model is {sum(f1_withoutVG)/len(f1_withoutVG)}")  

Macro F1 score of the trained model is 0.5039813566257582


#### Leaving the WD dataset and testing on it

In [96]:
df_positive=res_WD[res_WD["label"]==1]
df_negative=res_WD[res_WD["label"]==0]
print(len(df_negative))
print(len(df_positive))

22425
4589


In [97]:
f1_withoutWD=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_WD[res_WD["label"]==1]
    df_negative=res_WD[res_WD["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_WD['sentence'].values
    y_test=df_WD['label'].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutWD.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
9178/9178 [==============================] - 21s 2ms/step - loss: 0.6381 - accuracy: 0.6270
Epoch 2/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.4992 - accuracy: 0.7614
Epoch 3/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.2588 - accuracy: 0.8990
Epoch 4/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.1028 - accuracy: 0.9655
Epoch 5/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.0550 - accuracy: 0.9830
Epoch 6/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.0379 - accuracy: 0.9874
Epoch 7/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.0337 - accuracy: 0.9887
Epoch 8/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.0288 - accuracy: 0.9907
Epoch 9/10
9178/9178 [==============================] - 20s 2ms/step - loss: 0.0259 - accuracy: 0.9908
Epoch 10/10
3872/3872 [==============================] - 5s 1ms/s

In [98]:
print(f"Macro F1 score of the trained model is {sum(f1_withoutWD)/len(f1_withoutWD)}")  

Macro F1 score of the trained model is 0.423025331419166


#### Leaving the WTP dataset and testing on it

In [99]:
df_positive=res_WTP[res_WTP["label"]==1]
df_negative=res_WTP[res_WTP["label"]==0]
print(len(df_negative))
print(len(df_positive))

18811
3665


In [100]:
f1_withoutWTP=[]
for i in range(0,10):
    print(f"Model {i}")
    df_positive=res_WTP[res_WTP["label"]==1]
    df_negative=res_WTP[res_WTP["label"]==0]
    df_negative=df_negative.sample(len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_WTP['sentence'].values
    y_test=df_WTP['label'].values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences)

    X_train=tokenizer.texts_to_sequences(sentences)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=100

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)


    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    print(classification_report(y_test, y_pred.round()))
    report = classification_report(y_test, y_pred.round(),output_dict=True)
    f1_withoutWTP.append(report["macro avg"]["f1-score"])

Model 0
Epoch 1/10
7330/7330 [==============================] - 16s 2ms/step - loss: 0.6157 - accuracy: 0.6566
Epoch 2/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.4597 - accuracy: 0.7956
Epoch 3/10
7330/7330 [==============================] - 14s 2ms/step - loss: 0.2453 - accuracy: 0.9027
Epoch 4/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.1013 - accuracy: 0.9656
Epoch 5/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.0534 - accuracy: 0.9831
Epoch 6/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.0363 - accuracy: 0.9900
Epoch 7/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.0296 - accuracy: 0.9918
Epoch 8/10
7330/7330 [==============================] - 14s 2ms/step - loss: 0.0241 - accuracy: 0.9930
Epoch 9/10
7330/7330 [==============================] - 15s 2ms/step - loss: 0.0216 - accuracy: 0.9928
Epoch 10/10
8410/8410 [==============================] - 6s 761us

In [101]:
print(f"The Macro F1-Score on the MT dataset is {sum(f1_withoutWTP)/len(f1_withoutWTP)}")

The Macro F1-Score on the MT dataset is 0.4773337028515736
